In [1]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

/home/sujalnath/dev/emotion-gpt/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Load predictions and Dataset

In [4]:
pred_df = pd.read_csv("test_predictions.csv")

pred_df = pred_df[['text', 'belong', 'belong_pred', 'burden', 'burden_pred']]
print("Loaded data: ", pred_df.shape)
print(pred_df.head())

Loaded data:  (1057, 5)
                                                text  belong  belong_pred  \
0  [I'm not seeking any medical advice, just want...       0            0   
1  Im afraid. The suicide scale is true. Its gets...       1            1   
2  I have found that one of the main issues keepi...       1            1   
3  :(( not sure wats wrong with me. i think i‚Äôv...       1            1   
4  Help? I'm just feeling really shitty at how I ...       0            0   

   burden  burden_pred  
0       0            0  
1       1            1  
2       1            1  
3       1            1  
4       0            0  


### Choosing a Free Explanation Model

We'll use a T5 model from Hugging Face (lightweight and free)

It can generate short explanations from prompts

In [5]:
model_name = 'google/flan-t5-base'

print(f"\nLoading model: {model_name} ...")
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


Loading model: google/flan-t5-base ...


### Define Explanation Function

In [8]:
def generate_explanation(text, label_type, prediction):
    if label_type == 'belong':
        label_text = 'feels belongingness' if prediction == 1 else 'does not feel belongingness'
    else:
        label_text = 'feels burdened' if prediction == 1 else 'does not feel burdened'

    prompt = f"Text: {text}\nExplain why the person {label_text}."
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)
    outputs = model.generate(**inputs, max_new_tokens=60, temperature=0.7, top_p=0.9)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

### Generate Explanations

In [9]:
print("\nGenerating explanations... (this may take a few minutes)")
explanations = []

for i, row in pred_df.iterrows():
    text = row['text'][:400] # limit to avoid overly long input
    expl_belong = generate_explanation(text, 'belong', row['belong_pred'])
    expl_burden = generate_explanation(text, 'burden', row['burden_pred'])
    explanations.append({
        'text': text,
        'belong_pred': row['belong_pred'],
        'belong_explanation': expl_belong,
        'burden_pred': row['burden_pred'],
        'burden_explanation': expl_burden
    })

expl_df = pd.DataFrame(explanations)
expl_df.to_csv('generated_explanations.csv', index=False)

print("\n✅ Explanations generated and saved to 'generated_explanations.csv'")

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



Generating explanations... (this may take a few minutes)

✅ Explanations generated and saved to 'generated_explanations.csv'


### View Samples

In [10]:
print("\nSample Generated Explanations:")
print(expl_df.sample(5))


Sample Generated Explanations:
                                                  text  belong_pred  \
326  I just need some interaction, I‚Äôm trying rea...            1   
269  I feel as if depression has turned me into som...            1   
173  I've been depressed since I was 18 and I'm 26 ...            1   
542  So I got a psychosis and it really sucked, it ...            0   
645  I feel lonely because it's another NYE and it ...            1   

                                    belong_explanation  burden_pred  \
326                                 They are a friend.            1   
269                                  They are a friend            0   
173  I would have 30min sessions, Explain why the p...            0   
542                              They are not a person            0   
645      They feel lonely because they have no friends            0   

                                    burden_explanation  
326                        They are addicted to drugs.  
